This notebook animates one of the human-MRP interactions for the study.

In [1]:
import rospy
from geometry_msgs.msg import Twist

import tf
from gazebo_msgs.msg import ModelState
from gazebo_msgs.srv import GetModelState
from gazebo_msgs.srv import SetModelState

In [2]:
get_state = rospy.ServiceProxy('/gazebo/get_model_state', GetModelState)
set_state = rospy.ServiceProxy('/gazebo/set_model_state', SetModelState)

## Helper Functions

In [3]:
def get_model_state_from_response(model_name, model_state_response):
    new_model_state = ModelState()
    new_model_state.model_name = model_name
    
    # copy over post and orientation
    new_model_state.pose = model_state_response.pose
    new_model_state.twist = model_state_response.twist
    
    return new_model_state

In [4]:
def get_world_state():
    models = {
        'tb_sim': None,
        'local_user': None,
        'elevator_door_left': None,
        'elevator_door_right': None
    }

    rospy.wait_for_service('/gazebo/get_model_state')

    try:
        for model_name in models.keys():
            model_msg = get_state(model_name, 'world')
            
            assert model_msg.success is True
            
            models[model_name] = get_model_state_from_response(
                model_name, model_msg)

    except rospy.ServiceException:
        print('/gazebo/get_model_state service call failed')
    
    return models

In [5]:
def reset_world(models):
    rospy.wait_for_service('/gazebo/set_model_state')
    
    try:
        for model_name, model_state in models.items():
            set_state = rospy.ServiceProxy('/gazebo/set_model_state', SetModelState)
            
            set_state(model_state)
            
    except rospy.ServiceException:
        print ("Service call failed: %s" % str(rospy.ServiceException))

## Start Simulation

In [6]:
# before starting simulation, save the initial world state
initial_world_state = get_world_state()

In [7]:
def move_elevator_doors(direction):
    models = get_world_state()
    delta_t = direction * 0.01

    left_door_state = models['elevator_door_left']
    right_door_state = models['elevator_door_right']

    for _ in range(100):
        left_door_state.pose.position.y -= delta_t
        right_door_state.pose.position.y += delta_t

        rospy.wait_for_service('/gazebo/set_model_state')
        try:
            set_state = rospy.ServiceProxy('/gazebo/set_model_state', SetModelState)
            set_state(left_door_state)
            set_state(right_door_state)

        except rospy.ServiceException:
            print ("Service call failed: %s" % str(rospy.ServiceException))

Elevator arrives.

In [32]:
move_elevator_doors(1)

Move MRP into elevator.

In [9]:
rospy.init_node('test_node', anonymous=True)
pub = rospy.Publisher('/tb_cmd_vel', Twist, queue_size=10)

In [35]:
message = Twist()

message.linear.x = 0.2

pub.publish(message)

In [36]:
message = Twist()

message.linear.x = 0.2
message.angular.z = 0.2

pub.publish(message)

In [39]:
message = Twist()

message.angular.z = 0.7

pub.publish(message)

In [40]:
message = Twist()

pub.publish(message)

In [41]:
move_elevator_doors(-1)

Close elevator doors.

In [31]:
reset_world(initial_world_state)